<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/Chalange_Omega3/Travelling_Salesperson_Problems_version_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descrição do problema

Este código faz uma formulação de TSP que identifiqua a rota mais curta para as capitais brasileiras, que vai a todas as cidades uma vez e retorna à cidade de origem.

O Problema do Caixeiro Viajante (Traveling Salesperson - TSP) é um dos mais famosos problemas de otimização combinatória. Este problema é fácil de explicar, mas complicado de resolver - mesmo para casos com um pequeno número de cidades.

-------------------------------------------------------

O TSP pode ser definido da seguinte forma: para uma determinada lista de cidades e as distâncias entre cada par delas, queremos encontrar a rota mais curta possível, que vai a cada cidade uma vez e retorna à cidade de origem.

Uma possível formulação para esse problema seria:

* Indices: i, j = cidades
* Parâmetros: Cij = Custo (distância) para ir da cidade i para a cidade j. 
* Variáveis: Xij = 1 se a pessoa se dirigir da cidade i para a cidade j;  0 caso contrário.
* Função objetivo: Min Somatório em  ij de (Cij*Xij); que minimiza a distância total percorrida.
* Restrições: 

   a) Somatório em j de Xij =1 para todo i (de uma cidade i pode-se ir apenas para uma única cidade j.

   b) Somatório em i de Xij =1 para todo j (de uma cidade j pode-se ir apenas para uma única cidade i.

* Hípoteses: Os dados usados consideram que, para todas as cidades, a distância para ir da cidade A até a cidade B é a mesma para ir da cidade B até a cidade A.


--------------------------------------------------------------------


O problem foi resolvido usando o OR-Tools, uma biblioteca de otimização open source desenvolvida pelo Google. Escolheu-se essa ferramenta pois a mesma foi criada para lidar com problemas difíceis de aplicação prática, entre eles o roteamento de veículos, problemas de programação linear e inteira, e programação com restrições. Os modelos podem ser escrito em diversas linguaguens e resolvidos através de vários solvers. Além disso, é possível usar a Google Maps Distance Matrix API, a qual possibilita criar dinamicamente uma matriz de distância para qualquer conjunto de locais definidos por endereços ou por latitudes e longitudes, podendo assim ser usada para solucionar vários problemas reais de roteamento. Essa biblioteca já contém o TSP formulado, então, a formulação sugeriada acima não precisou ser implementada.

-------------------------
Autor: Raiana Roland Seixas
20/07/2022

# Instalar e importar bibliotecas e carregar dados de entrada

In [14]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
# Importar Bibliotecas
import pandas as pd
import plotly.express as px # plotar mapa com a rota 
import io

''' Travelling Salesperson Problem (TSP) between cities.'''
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [16]:
# Obter dados do Github
!git clone https://github.com/RaianaSeixas/Data_Science_Otimization_OR.git
%cd Data_Science_Otimization_OR/Data

fatal: destination path 'Data_Science_Otimization_OR' already exists and is not an empty directory.
/content/Data_Science_Otimization_OR/Data/Data_Science_Otimization_OR/Data


In [17]:
'''
Foram disponibilizados 3 datasets para o problema em questão.  
Nesta célula pode-se escolher com qual deles se deseja trabalhar.
Meus comentários serão baseados nos resultados obtidos com o dataset states_line, porém pode-se rodar o problema com as 3 opções.
Escolhi este dataset, pois  em todos conjuntos de dados, as distâncias entre as ciades são simétricas. 
Assim, este é o conjunto mais completo que considera o mesmo critério para medida de distância. 
'''
#dataset_name ='states_gmaps.csv'  # distância em km entre as capitais brasileiras (informações do Google). Dados ausentes para Amapá e Acre
dataset_name ='states_line.csv'  # distância em km de entre todas as capitais assumindo uma linha conectando-as
#dataset_name ='states_merged.csv' # dataset artificial no qual os dados ausentes do Google para Amapá e Acre foram substituídos pela distância considerando uma linha 

In [18]:
df = pd.DataFrame(pd.read_csv(dataset_name))
df.head()

,São Paulo,Rio de Janeiro,Minas Gerais,Rio Grande do Sul,Pernambuco,Ceará,Bahia,Paraná,Pará,Goiás,...,Paraíba,Piauí,Mato Grosso,Mato Grosso do Sul,Sergipe,Amapá,Rondônia,Acre,Tocantins,Roraima
0,0.000000,354.557321,492.561745,852.763702,2126.951268,2366.805644,1457.532585,342.443520,2467.265855,810.258174,...,2219.374772,2094.675477,1330.507665,893.525598,1731.087701,2667.276248,2468.533941,2708.559270,1492.162778,3304.536603
1,354.557321,0.000000,342.337926,1121.072276,1875.035468,2190.647081,1215.465687,677.792623,2455.146750,938.003328,...,1973.855227,1983.803733,1577.605256,1208.758382,1486.300610,2690.286145,2710.242987,2984.087454,1510.388182,3431.919571
2,492.561745,342.337926,0.000000,1344.567917,1635.086515,1888.834505,965.600360,826.013748,2113.032483,670.440922,...,1726.940870,1652.997918,1376.313628,1118.632967,1238.956225,2351.045347,2482.002571,2790.495163,1174.123563,3120.769148
3,852.763702,1121.072276,1344.567917,0.000000,2975.531406,3212.503748,2306.960107,547.212395,3194.128093,1495.239654,...,3069.619614,2912.951307,1686.118439,1121.554669,2580.662593,3346.164787,2712.819615,2820.369451,2224.310608,3791.702726
4,2126.951268,1875.035468,1635.086515,2975.531406,0.000000,629.108539,669.488856,2460.033424,1673.663528,1833.172567,...,108.480926,929.975492,2448.546265,2526.112206,396.445424,2003.187408,3189.355451,3617.166405,1487.819797,3103.051463


In [19]:
  ''' Automatização de 1 único código, sendo possível escolher o dataset  '''
  
  # Ajustes foram feitos para  Dataset que considera as distâncias do Google. Esse dataset tem dados ausentes para 2 estados. 
  
  maps_data = pd.DataFrame(pd.read_csv('states_coords.csv'))
  if dataset_name =='states_gmaps.csv':
    # Remover colunas que contém dados relativos ao Acre e Amapá da dataset states_merged
    dataset_name ='states_merged.csv'
    df = pd.DataFrame(pd.read_csv(dataset_name))
    df.drop(['Amapá', 'Acre'], axis=1, inplace=True)
    df.drop([21, 23], axis=0, inplace=True)
    nomes = df.columns
    maps_data.drop([21, 23], axis=0, inplace=True)
    maps_data = maps_data.reset_index(drop=True)
   
  else:
    df = pd.DataFrame(pd.read_csv(dataset_name))
    nomes = df.columns

  dataset=df

# Modelo

### Código fonte


*   Foi implementada uma adaptação do Problema de Roteamento de Veículos (VRP), cujo o código está descrito em:

 https://developers.google.com/optimization/routing/vrp
 
 No Problema de Roteamento de Veículos (VRP), o objetivo é encontrar rotas ideais para vários veículos que visitam um conjunto de locais. (Quando há apenas um veículo, reduz-se ao Problema do Caixeiro Viajante).


In [20]:
# As funções abaixo foram copiadas das instuções para usar a biblioteca OR-Tools. Apenas ajustes menores foram realizados.

def create_data_model(partida,dataset):
  """Stores the data for the problem."""
  df = dataset # conjunto de dados usado
  vals = df.values.tolist() # converte os valores do dataframe para lista 
  data = {} # cria o dicionario que representa a entrada do método
  data['distance_matrix'] = vals
  data['num_vehicles'] = 1 # número de veículos em uso
  data['depot'] = partida # ponto de partida
  return data

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    
    return data['distance_matrix'][from_node][to_node]

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Função Objetivo: {} km'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Rota percorrida:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {} km\n'.format(route_distance)


In [21]:
'''Resolução considerando cada 1 das cidades como ponto de origem '''

# Notou-se que ao mudar o ponto de partida, os valores de função objetivo podiam mudar, 
# Assim, como o tempo de execução era pequeno, resolveu-se o problema considerando como início cada uma das cidades. 

Num = df.shape[1] # número de localidades
best_distance = 1e99
best_choice = 1e99
best_object =[]

for i in range(Num):
  partida = i

  """Entry point of the program."""
  # Instantiate the data problem.
  data = create_data_model(partida,dataset)

  # Create the routing index manager.
  manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                            data['num_vehicles'], data['depot'])
  # len(data['distance_matrix'])= number of locations

  # Create Routing Model.
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)

  # Define cost of each arc.
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  # Setting first solution heuristic.
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (
  routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  # Solve the problem.
  solution = routing.SolveWithParameters(search_parameters)

  # Print solution on console.
  #if solution:
  #  print_solution(manager, routing, solution)
  distancia = solution.ObjectiveValue()
  print('Partindo de: ',nomes[i],'   ->    Distância: ', distancia,'km')
  if(i==0):
    best_object.append(manager)
    best_object.append(routing)
    best_object.append(solution)
  if(best_distance >= distancia):
    best_distance = distancia
    best_choice = i
    best_object[0] = manager
    best_object[1] = routing
    best_object[2] = solution

Partindo de:  São Paulo    ->    Distância:  13706 km
Partindo de:  Rio de Janeiro    ->    Distância:  13862 km
Partindo de:  Minas Gerais    ->    Distância:  13862 km
Partindo de:  Rio Grande do Sul    ->    Distância:  13862 km
Partindo de:  Pernambuco    ->    Distância:  13862 km
Partindo de:  Ceará    ->    Distância:  13862 km
Partindo de:  Bahia    ->    Distância:  13706 km
Partindo de:  Paraná    ->    Distância:  13862 km
Partindo de:  Pará    ->    Distância:  13862 km
Partindo de:  Goiás    ->    Distância:  13706 km
Partindo de:  Amazonas    ->    Distância:  13862 km
Partindo de:  Espírito Santo    ->    Distância:  13862 km
Partindo de:  Alagoas    ->    Distância:  13862 km
Partindo de:  Rio Grande do Norte    ->    Distância:  13862 km
Partindo de:  Maranhão    ->    Distância:  13862 km
Partindo de:  Santa Catarina    ->    Distância:  13862 km
Partindo de:  Paraíba    ->    Distância:  13862 km
Partindo de:  Piauí    ->    Distância:  13862 km
Partindo de:  Mato Gr

## Resposta otimizada do modelo

In [22]:
print('Melhor partida =',nomes[best_choice])
print('Distância total percorrida =',best_distance,'km')

Melhor partida = Tocantins
Distância total percorrida = 13706 km


Pode-se perceber que, há várias cidades que podem servir como melhor ponto de partida, retornando o mesmo valor de função objetivo.

In [23]:
''' Impressão da sequência de cidades percorridas '''

manager = best_object[0] 
routing = best_object[1] 
solution = best_object[2] 

index = routing.Start(0)
plan_output = []

while not routing.IsEnd(index):
  plan_output.append(manager.IndexToNode(index))
  index = solution.Value(routing.NextVar(index))

plan_output.append(best_choice)

print_solution(manager, routing, solution)

Função Objetivo: 13706 km
Rota percorrida:
 24 -> 9 -> 2 -> 11 -> 1 -> 0 -> 7 -> 15 -> 3 -> 19 -> 18 -> 23 -> 22 -> 10 -> 25 -> 21 -> 8 -> 14 -> 17 -> 5 -> 13 -> 16 -> 4 -> 12 -> 20 -> 6 -> 24



## Mapa da solução

In [24]:
solution_map = plan_output

In [25]:
#maps_data = pd.DataFrame(pd.read_csv('states_coords.csv'))
coord = maps_data[['lat','lng']]
#coord

In [26]:
fig = px.scatter_mapbox(coord, lat="lat", lon="lng",
                        hover_name=coord.index, zoom=3, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

tsp =solution_map

fig.add_traces(px.line_mapbox(coord.loc[tsp], lat="lat", lon="lng").data)

# Sugestão trabalhos futuros

* O solver foi definido de modo automático, esse é o parâmetro padrão do método, o qual escolhe o solve de acordo com as características do problema. Seria interessante testar os diferentes solvers disponíveis. Há outros parâmetros que poderiam ter sido explorados, como por exemplo, em termos da definição do nó inicial (usei força bruta resolvendo o problema considerando cada um dos nós, o que não seria viável em um problema maior). Mais informações sobre definição de parâmetros em : https://developers.google.com/optimization/routing/routing_options

* Seria interessante tentar resolver esse problema usando os dados reais atualizados de distância através da Google Maps Distance Matrix API. Mais informações em: https://developers.google.com/optimization/routing/google_direction

* Também seria interessante resolver problemas de roteamento de veículos para estas cidades, considerando vários veículos, restrições de capacidade, de coleta e entrega, e de janelas temporais. Essa biblioteca tem módulos para incluir esses tipos de restrições.

